<a href="https://colab.research.google.com/github/rhysdavies21/library/blob/master/book_sabr_%26_sabr_lmm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SABR and SABR LIBOR Market Models in Practice --- Crispoldi, Wigger & Larkin##

In [1]:
# Black formula
# Reference SABR & SABR LIBOR Market Models, page 32, table 4.1
# Note that this is NOT the discounted price

from scipy.stats import norm
import math

def Black(F_0, y, expiry, vol, isCall):
  '''
  Compute the Black formula.
  @var F_0: Forward rate at time 0
  @var y: option strike
  @var expiry: option expiry (in years)
  @var vol: Black implied volatility
  @var isCall: True or False
  '''
  option_value = 0
  if expiry * vol == 0.0:
    if isCall: 
      option_value = max(F_0 - y, 0.0)
    else:
      option_value = max(y - F_0, 0.0)
  else:
    d1 = dPlusBlack(F_0 = F_0, y = y, expiry = expiry, vol = vol)
    d2 = dMinusBlack(F_0 = F_0, y = y, expiry = expiry, vol = vol)
    if isCall:
      option_value = (F_0 * norm.cdf(d1) - y * norm.cdf(d2))
    else:
      option_value = (y * norm.cdf(-d2) - F_0 * norm.cdf(-d1))
  return option_value


def dPlusBlack(F_0, y, expiry, vol):
  '''
  Compute the d+ term appearing in the Black formula
  @var F_0: forward rate at time 0
  @vary y: option strike
  @var expiry: option expiry (in years)
  @var vol: Black implied volatility
  '''
  d_plus =  ((math.log(F_0 / y) + 0.5 * vol * vol * expiry) / (vol * math.sqrt(expiry)))
  return d_plus


def dMinusBlack(F_0, y, expiry, vol):
  '''
  Compute the d- term appearing in the Black formula
  @var F_0: forward rate at time 0
  @vary y: option strike
  @var expiry: option expiry (in years)
  @var vol: Black implied volatility
  '''
  d_minus =  (dPlusBlack(F_0 = F_0, y = y, expiry = expiry, vol = vol) - vol * math.sqrt(expiry))
  return d_minus

In [2]:
# Run Black (undiscounted) formula

F_0 = 0.02 
y = 0.03
expiry =  0.5
vol = 0.4
isCall = 1

a = Black(F_0, y, expiry, vol, isCall)
b = dPlusBlack(F_0, y, expiry, vol)
c = dMinusBlack(F_0, y, expiry, vol)

print('BOOK Black (undiscounted) price: ', round(a,10))
print('BOOK dPlusBlack: ', b)
print('BOOK dMinusBlack', c)

BOOK Black (undiscounted) price:  0.0002341801
BOOK dPlusBlack:  -1.2921142811517878
BOOK dMinusBlack -1.574956993626407


In [3]:
# First-order derivative of a function using central difference 
# Reference SABR & SABR LIBOR Market Models, page 41, table 4.6

def computeFirstDerivative(v_u_plus_du, v_u_minus_du, du):
  '''
  Compute the first derivative of a function using central difference

  @var v_u_plus_du: is the value of the function computed for a positive bump amount du
  @var v_u_minus_du: is the value of the function computed for a negative bump amount du
  @var du: bump amount
  '''
  first_derivative = (v_u_plus_du - v_u_minus_du) / (2.0 * du)

  return first_derivative

In [4]:
# First-order derivative of a function using central difference 
# Reference SABR & SABR LIBOR Market Models, page 41, table 4.6

def computeSecondDerivative(v_u, v_u_plus_du, v_u_minus_du, du):
  '''
  Compute the second derivative of a function using central difference

  @var v_u: is the value of the function
  @var v_u_plus_du: is the value of the function computed for a positive bump amount du
  @var v_u_minus_du: is the value of the function computed for a negative bump amount du
  @var du: bump amount
  '''
  second_derivative = (v_u_plus_du - 2.0*v_u + v_u_minus_du) / (du * du)

  return second_derivative

In [5]:
# Hagan et al. lognormal approximation
# Reference SABR & SABR LIBOR Market Models, page 58, table 5.1

import math
def haganLogNormalApprox(y, expiry, F_0, alpha_0, beta, nu, rho):
  '''
  Function which returns the Black implied volatility, computed using the 
  Hagan et al. lognormal approximations

  @var y: option strike
  @expiry: option expiry (in years)
  @var F_0: forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  '''
  one_beta = 1.0 - beta
  one_betasqr =  one_beta * one_beta
  if F_0 != y:
    fK = F_0 * y
    fK_beta = math.pow(fK, one_beta / 2.0)
    log_fK = math.log(F_0 / y)
    z = nu / alpha_0 * fK_beta *log_fK
    x = math.log((math.sqrt(1.0 - 2.0 * rho * 
                            z + z * z) + z - rho) / (1 - rho))
    sigma_1 = (alpha_0 / fK_beta / (1.0 + one_betasqr / 
              24.0 * log_fK * log_fK +
              math.pow(one_beta * log_fK, 4) / 1920.0) *
              (z / x))
    sigma_exp = (one_betasqr / 24.0 * alpha_0 * alpha_0 /
                 fK_beta / fK_beta + 0.25 * rho * beta *
                 nu * alpha_0 / fK_beta + 
                 (2.0 - 3.0 * rho * rho) / 24.0 * nu * nu)
    sigma = sigma_1 * (1.0 + sigma_exp * expiry)
  else:
    f_beta = math.pow(F_0, one_beta)
    f_two_beta = math.pow(F_0, (2.0 - 2.0 * beta))
    sigma - ((alpha_0 / f_beta) * (1.0 + 
                                   ((one_betasqr / 24.0) *
                                    (alpha_0 * alpha_0 / f_two_beta) + 
                                    (0.25 * rho * beta * nu * alpha_0 / f_beta) +
                                    24.0 * nu * nu) * expiry))
  return sigma

In [6]:
# Run Hagan et al. lognormal approximation

y = 0.09
expiry = 3
F_0 = 0.03
alpha_0 = 0.2
beta = .2
nu = 0.5
rho = 0.5

x = haganLogNormalApprox(y, expiry, F_0, alpha_0, beta, nu, rho)
print('SABR Vol: ', round(x, 10))

SABR Vol:  3.272972092


In [7]:
# Run SABR calculation example

y = 0.02
expiry = 0.5
F_0 = 0.04
alpha_0 = 0.2
beta = .8
nu = 0.3
rho = -0.4

x = haganLogNormalApprox(y, expiry, F_0, alpha_0, beta, nu, rho)
round(x, 10)

0.4584267891

In [8]:
# SABR delta computation 
# Reference SABR & SABR LIBOR Market Models, page 74, table 5.2

def computeSABRDelta(y, expiry, F_0, alpha_0, beta, rho, nu, isCall):
  '''
  Compute the SABR delta

  @var y: option strike
  @var expiry: option expiry (in years)
  @var F_0: forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  @var isCall: True or False
  '''
  small_figure = 1e-6
  
  F_0_plus_h = F_0 + small_figure
  avg_alpha_plus = (alpha_0 + (rho * nu / math.pow(F_0, beta)) * small_figure)
  vol = haganLogNormalApprox(y, expiry, F_0_plus_h, avg_alpha_plus, beta, nu, rho)
  px_f_plus_h = black(F_0_plus_h, y, expiry, vol, isCall)

  F_0_minus_h = F_0 - small_figure
  avg_alpha_minus = (alpha_0 + (rho * nu / math.pow(F_0, beta)) * (-small_figure))
  vol = haganLogNormalApprox(y, expiry, F_0_minus_h, avg_alpha_minus, beta, nu, rho)
  px_f_minus_h = black(F_0_minus_h, y, expiry, vol, isCall)
  sabr_delta = computeFirstDerivative(px_f_plus_h, px_f_minus_h, small_figure)

  return sabr_delta

In [9]:
# SABR vega computation 
# Reference SABR & SABR LIBOR Market Models, page 76, table 5.3

import math

def computeSABRVega(y, expiry, F_0, alpha_0, beta, rho, nu, isCall):
  '''
  Compute the SABR vega

  @var y: option strike
  @var expiry: option expiry (in years)
  @var F_0: forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  @var isCall: True or False
  '''
  small_figure = 1e-6
  alpha_plus_h = alpha_0 + small_figure
  avg_F_plus = (F_0 + (rho * math.pow(F_0, beta) / nu) * small_figure)
  vol_plus = haganLogNormalApprox(y, expiry, avg_F_plus, alpha_plus_h, beta, nu, rho)
  px_a_plus_h = black(F_0, y, expiry, vol_plus, isCall)

  alpha_minus_h = alpha_0 - small_figure
  avg_F_minus = (F_0 + (rho * math.pow(F_0, beta) / nu) * (-small_figure))
  vol_minus = haganLogNormalApprox(y, expiry, avg_F_minus, alpha_minus_h, beta, nu, rho)
  px_a_minus_h = black(F_0, y, expiry, vol_minus, isCall)

  sabr_vega = computeFirstDerivative(px_a_plus_h, px_a_minus_h, small_figure)

  return sabr_vega

In [10]:
# Draw two correlated random numbers
# Reference SABR & SABR LIBOR Market Models, page 80, table 5.4

import random
# Seed the random number generator
random.seed()
import math

def drawTwoRandomNumbers(rho):
  '''
  Draw a pair of correlated random numbers
  @var rho: SABR Rho
  '''
  rand_list = []
  z1 = random.gauss(0,1)
  y1 = random.gauss(0,1)
  rand_list.append(z1)
  term1 = z1 * rho
  term2 = (y1 * math.pow((1.0 - math.pow(rho, 2.0)), 0.5))
  x2 = term1 + term2
  rand_list.append(x2)

  return rand_list

In [11]:
# Monte Carlo Euler scheme
# Reference SABR & SABR LIBOR Market Models, page 82, table 5.5

import math
def simulateSABRMonteCarloEuler(no_of_sim, no_of_steps, 
                                expiry, F_0, alpha_0, beta, rho, nu):
  '''
  Monte Carlo SABR using Euler scheme
  @var no_of_sim: Monte Carlo paths
  @var no_of_steps: discretization steps required
  to reach the option expiry date
  @var expiry: optio expiry (in years)
  @var F_0: forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  '''
  # Step length in years
  dt = float(expiry) / float(no_of_steps)
  dt_sqrt = math.sqrt(dt)
  no_of_sim_counter = 0
  simulated_forwards = []
  while no_of_sim_counter < no_of_sim:
    F_t = F_0
    alpha_t = alpha_0
    no_of_steps_counter = 1
    while no_of_steps_counter <= no_of_steps:
      # Zero absorbing boundary used for all the beta
      # choices except beta = 0 and beta = 1
      if ((beta > 0 and beta < 1) and F_t <= 0):
        F_t = 0
        no_of_steps_counter =  no_of_steps + 1
      else:
        # Generate two correlated random numbers
        rand = drawTwoRandomNumbers(rho)
        # Simulate the forward interest rate using the Euler
        # scheme. Use the absolute for the diffusion to avoid 
        # numerical issues if the forward interest rate goes into 
        # negative territory
        dW_F = dt_sqrt * rand[0]
        F_b = math.pow(abs(F_t), beta)
        F_t = F_t + alpha_t * F_b * dW_F
        # Simulate the stochastic volatility using the Euler scheme
        dW_a = dt_sqrt * rand[1]
        alpha_t = (alpha_t + nu * alpha_t * dW_a)
      no_of_steps_counter += 1
    # At the end of each path, we store the forward interest rate in a list
    simulated_forwards.append(F_t)
    no_of_sim_counter = no_of_sim_counter + 1
  return simulated_forwards

In [12]:
# Run Monte Carlo Euler scheme example

no_of_sim = 10000
no_of_steps = 100
expiry = 1
F_0 = 0.05
alpha_0 = 0.2
beta = 0.6
rho = -0.3
nu = 0.2

see_paths_euler = simulateSABRMonteCarloEuler(no_of_sim, no_of_steps, expiry, F_0, alpha_0, beta, rho, nu)
print('See first 5 simulations: ', see_paths_euler[0:5])

See first 5 simulations:  [0.06641095469586905, 0.03920072290398694, 0.1252632499822832, 0.005986332335048465, 0.041169573755243986]


In [13]:
# Monte Carlo Milstein scheme for SABR
# Reference SABR & SABR LIBOR Market Models, page 89, table 5.10

import math
def simulateSABRMonteCarloMilstein(no_of_sim, no_of_steps, expiry, F_0, 
                                   alpha_0, beta, rho, nu):
  '''
  Monte Carlo SABR using Milstein scheme
  @var no_of_sim: Monte Carlo paths
  @var no_of_steps: discretization steps required
  to reach the option expiry date
  @var expiry: option expiry (in years)
  @var F_0: Forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  '''
  # Step length in years
  dt = float(expiry) / float(no_of_steps)
  dt_sqrt = math.sqrt(dt)
  no_of_sim_counter = 0
  simulated_forwards = []
  while no_of_sim_counter < no_of_sim:
    F_t = F_0
    alpha_t = alpha_0
    no_of_steps_counter = 1
    while no_of_steps_counter <= no_of_steps:
      # Zero absorbing boundary used for all the beta choices
      # except beta = 0 and beta = 1
      if ((beta > 0 and beta < 1) and F_t <= 0):
        F_t = 0
        no_of_steps_counter = no_of_steps + 1
      else:
        # Generate two correlated random numbers
        rand = drawTwoRandomNumbers(rho)
        # Simulate the forward interest rate using the Milstein scheme. Use
        # the absolute for the diffusion to avoid the numerical issues if the
        # forward interest rate goes into negative territory
        dW_F = dt_sqrt * rand[0]
        F_b = math.pow(abs(F_t), beta)
        exp_F = 2.0 * beta - 1.0
        F_t = (F_t + alpha_t * F_b * dW_F 
               + 0.5 * beta * math.pow(alpha_t, 2.0) 
               * math.pow(abs(F_t), exp_F) * (rand[0] * rand[0] - 1.0) * dt )
        # Simulate the stochastic volatility using the Milstein scheme
        dW_a = dt_sqrt * rand[1]
        nu_sqr = math.pow(nu, 2.0)
        alpha_t = (alpha_t + nu * alpha_t * dW_a 
                   + 0.5 * nu_sqr * alpha_t * (rand[1] * rand[1] - 1.0) * dt)
      no_of_steps_counter += 1
    # At the end of each path, we store the forward interest rate in a list
    simulated_forwards.append(F_t)
    no_of_sim_counter = no_of_sim_counter + 1
  return simulated_forwards

In [14]:
# Run Monte Carlo Milstein example

no_of_sim = 10000
no_of_steps = 100
expiry = 1
F_0 = 0.05
alpha_0 = 0.2
beta = 0.6
rho = -0.3
nu = 0.2

see_paths_milstein = simulateSABRMonteCarloMilstein(no_of_sim, no_of_steps, expiry, F_0, alpha_0, beta, rho, nu)
print('See first 5 simulations: ', see_paths_milstein[0:5])

See first 5 simulations:  [0.018714060170356022, 0.05741112711059576, 0.09465318623933672, 0.07515930735333055, 0.0591014652016041]


In [15]:
# Double exponential correlation parameterization
# Reference SABR & SABR LIBOR Market Models, page 135, table 6.4
# Note multiple errors in code from book

import numpy as np

def generateParametricCorrelationMatrix(alpha, beta, rho_inf, maturity_grid):
  '''
  Function which generates a correlation matrix using the double 
  parameterization

  @var alpha: alpha parameter
  @var beta: beta parameter
  @var rho_inf: rho_inf parameter
  @var maturity_grid: represents a list containing the firward rate maturities
  which we are going to model. For example, if we want to model the
  semi-annual forward rates maturing between 1 and 5 years from now, we will set:
  maturity_grid = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
  '''
  grid = len(maturity_grid)
  # Create a null correlation matrix
  corr_matrix = np.zeros((grid, grid))
  for i in range(grid):
    for j in range(grid):
      first_e = np.exp(-beta * abs(maturity_grid[i] 
                                   - maturity_grid[j]))
      second_e = np.exp(-alpha * min(maturity_grid[i], 
                                     maturity_grid[j]))
      corr_matrix[i,j] = (rho_inf + (1.0 - rho_inf) *
                          first_e * second_e) 
  return corr_matrix

In [16]:
# Generate parametric correlation matrix example

alpha = 0.2
beta = 0.6
rho_inf = -0.3
maturity_grid = [1, 2, 3, 4, 5]

corr = generateParametricCorrelationMatrix(alpha, beta, rho_inf, maturity_grid)
corr

array([[ 0.76434998,  0.28412765,  0.02057605, -0.12406413, -0.20344435],
       [ 0.28412765,  0.57141606,  0.17824327, -0.03753453, -0.15595589],
       [ 0.02057605,  0.17824327,  0.41345513,  0.09155248, -0.08511145],
       [-0.12406413, -0.03753453,  0.09155248,  0.28412765,  0.02057605],
       [-0.20344435, -0.15595589, -0.08511145,  0.02057605,  0.17824327]])

In [17]:
# Compute the Black volatility in terms of the instantaneous volatility
# parameterized using equation 6.26 on page 138
# Reference SABR & SABR LIBOR Market Models, page 139, table 6.6

import numpy as np

def getCapletVolatility(expiry, a, b, c, d):
  '''
  Return the caplet volatility at time t=0, computed in terms of the 
  instantaneous volatility function form proposed by Rebonato

  @var expiry: caplet expiry (in years)
  @var a: parameter a of Rebonato's instant. vol function
  @var b: parameter b of Rebonato's instant. vol function
  @var c: parameter c of Rebonato's instant. vol function
  @var d: parameter d of Rebonato's instant. vol function
  '''
  a_sqr = a * a
  b_sqr = b * b
  c_sqr = c * c
  d_sqr = d * d
  tau = expiry
  exp_term = np.exp(-c *  tau)
  exp_plus_term = np.exp(c *  tau)
  term1 = (-2.0 * c_sqr * (a_sqr + 4.0 * a * d * exp_plus_term 
                           - 2.0 * c * d_sqr * tau * exp_plus_term * exp_plus_term))
  term2 = (2.0 * b * c * (2.0 * a * c * tau + a 
                          + 4.0 * d * exp_plus_term * (c * tau + 1.0)))
  term3 = (b_sqr * (-(2.0 * c_sqr * tau * tau + 2.0 * c * tau + 1.0)))
  variance_expiry = ((1.0 / (4.0 * c * c * c)) * exp_term * exp_term * (term1 - term2 + term3))
  
  tau = 0
  exp_term = np.exp(-c * tau)
  exp_plus_term = np.exp(c * tau)
  term1 = (-2.0 * c_sqr * (a_sqr + 4.0 * a * d * exp_plus_term 
                           - 2.0 * c * d_sqr * tau * exp_plus_term * exp_plus_term))
  term2 = (2.0 * b * c * (2.0 * a * c * tau + a 
                          + 4.0 * d * exp_plus_term * (c * tau + 1.0)))
  term3 = (b_sqr * (-(2.0 * c_sqr * tau * tau + 2.0 * c * tau + 1.0)))
  variance_t0 = ((1.0 / (4.0 * c * c * c)) * exp_term * exp_term * (term1 - term2 + term3))
  
  variance = variance_expiry - variance_t0
  volatility = np.sqrt(variance / expiry)
  
  return volatility

In [18]:
# Compute the instantaneous volatility in terms of the parametric form
# Reference SABR & SABR LIBOR Market Models, page 141, table 6.7

import numpy as np

def getInstantaneousVolatility(t, expiry, a, b, c, d):
  '''
  Return the instantaneous volatility computed in terms of the parametric
  form proposed by Rebonato at a given time t

  @var t: time at which we want to compute the instantaneous volatility (in years)
  @var expiry: caplet expiry (in years)
  @var a: parameter a of Rebonato's instant. vol function
  @var b: parameter b of Rebonato's instant. vol function
  @var c: parameter c of Rebonato's instant. vol function
  @var d: parameter d of Rebonato's instant. vol function
  '''
  tau = expiry - t
  instantaneous_vol = (a + b * tau) * np.exp(-c * tau) + d

  return instantaneous_vol

In [20]:
# Reduce the correlation matrix rank through the eigenvalue zeroing tehcnique
# Reference SABR & SABR LIBOR Market Models, page 145, table 6.8

import numpy as np

def reduceRank(corr_matrix, no_of_factors):
  '''
  Function which reduces the correlation matrix rank by employing the
  eigenvalue zeroing technique

  @var corr_matrix: input correlation matrix containing the forward-forward
  correlations
  @var no_of_factors: number of factors that we want to employ
  '''
  B = None
  reduced_cov_matrix = None
  reduced_corr_matrix = np.zeros((corr_matrix.shape[0], corr_matrix.shape[0]))
  Q, lambda_, Q_inv = np.linalg.svd(corr_matrix)
  H = np.zeros((corr_matrix.shape[0], no_of_factors))
  for i in range(no_of_factors):
    H[i, i] = np.sqrt(lambda_[i])
  B = Q.dot(H)
  reduced_cov_matrix = B.dot(B.T)
  # Transform the covariance matrix into a correlation matrix
  for i in range(corr_matri_shape[0]):
    for j in range(corr_matrix.shape[0]):
      reduced_corr_matrix[i, j] = (reduced_cov_matrix[i, j] 
                                   / (np.sqrt(reduced_cov_matrix[i, i] 
                                              * reduced_cov_matrix[j, j])))
  return B, reduced_corr_matrix

In [22]:
# Draw correlated random numbers
# Reference SABR & SABR LIBOR Market Models, page 147, table 6.10

import numpy as np

# See the random number generator
np.random.seed()

def drawRandomNumbers(no_of_factors, cholesky):
  '''
  Draw a set of random numbers

  @var no_of_factors: number of factors that we want to employ
  @var cholesky: Cholesky decomposition  of the correlation matrix
  describing the correlation among the random variables to simulate
  '''

  if no_of_factors > 1:
    rand = np.random.normal(size = no_of_factors)
    return cholesky.dot(rand)
  else:
    return np.random.normal()

In [6]:
# Simulation of the SABR LIBOR Market Model under the terminal measure Q^N
# Reference SABR & SABR LIBOR Market Models, page 194, table 7.7

import math
import numpy as np

def simulateSABRLMM(no_of_sim, no_steps, 
                    maturity_grid, F_0, s_0, epsilon, 
                    corr_fwd_fwd_full_rank,
                    corr_vol_vol_full_rank,
                    no_of_factos, beta, tau,
                    a_g, b_g, c_g, d_g, a_h, b_h, c_h, d_h):
  '''
  Monte Carlo SABR LMM using Euler scheme
  
  @var no_of_sim: Monte Carlo paths
  @var no_of_steps: discretization steps required to reach the option expiry date
  @var maturity_grid: list containing the forward maturities (in years). E.g.
  [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
  @var F_0: list of forward rate values at time t=0
  @var s_0: list of correction terms s
  @var epsilon: list of correction terns epsilon
  @corr_fwd_fwd_full_rank: forward-forward full rank correlation matrix
  @var corr_vol_vol_full_rank: volatility-volatility full rank correlation matrix
  @var no_of_factors: number of factors which we want to use in the simulation
  @var beta: list of beta values
  @var tau: forward rate rolling period (in years)
  @var a_g: parameter a of the g function
  @var a_g: parameter b of the g function
  @var a_g: parameter c of the g function
  @var a_g: parameter d of the g function
  @var a_g: parameter a of the h function
  @var a_g: parameter b of the h function
  @var a_g: parameter c of the h function
  @var a_g: parameter d of the h function
  '''

  no_of_fwds = len(F_0)
  simulated_forwards = np.zeros((no_of_fwds, no_of_sim))
  # Get the Cholesky decomposition as well as the reduced rank matrix from the
  # full rank forward-forward correlation matrix
  cholesky_fwd_fwd, corr_fwd_fwd = reduceRank(corr_fwd_fwd_full_rank, no_of_factors)
  # Get the Cholesky decomposition as well as the reduced rank matrix from the
  # full rank volatility-volatility correlation matrix
  cholesky_vol_vol, corr_vol_vol = reduceRank(corr_vol_vol_full_rank, no_of_factors)
  # Initialize variables
  F_t = np.array([0.0] * no_of_fwds)
  alpha_t = np.array([0.0] * no_of_fwds)
  s_t = np.array([1.0] * no_of_fwds)
  ds_t = np.array([0.0] * no_of_fwds)
  # Get the kast LIBOR fixing time
  # It is the next to last element stored in the time grid
  last_maturity_index = len(maturity_grid) - 2
  last_maturity = maturity_grid[last_maturity_index]
  # Step length in years
  dt = last_maturity / float(no_of_steps)
  dt_sqrt = math.sqrt(dt)
  no_sim = 0
  while no_sim < no_of_sim:
    t = 0
    no_step = 1
    for fwd_k in range(0, no_of_fwds):
      s_t[fwd_k] = s_0[fwd_k]
      F_t[fwd_k] = F_0[fwd_k]
      ds_t[fwd_k] = 0.0
  # Simulate the processes along the steps composing a path
  while no_step <= no_of_steps:
    # The t variable will be used to get the parametric vol and vol of vol 
    # corresponding to the time step being simulated
    t = no_step * dt
    # Generate the correlated Brownian motions used to simulate dF and ds
    dW_F = drawRandomNumbers(no_of_factors, cholesky_fwd_fwd) * dt_sqrt
    dW_s = drawRandomNumbers(no_of_factors, cholesky_vol_vol) * dt_sqrt
  drift_F = 0.0
  shared_drift_part = [0.0] * no_of_fwds
  drift_correction = [0.0] * no_of_fwds
  # Simulate each forward rate starting from the last one
  for i in range(1, no_of_fwds):
    fwd_k = no_of_fwds - i 
    # The simulation is run only for forward rates which are still alive
    if t < maturity_grid[fwd_k - 1]: 
      g_t = getInstantaneousVolatility(t, maturity_grid[fwd_k - 1], a_g, b_g, c_g, d_g)
      h_t = getInstantaneousVolatility(t, maturity_grid[fwd_k - 1], a_h, b_h, c_h, d_h)
      F_beta_t = math.pow(abs(F_t[fwd_k]), beta[fwd_k])
      ds_t[fwd_k] = (h_t * epsilon[fwd_k] * s_t[fwd_k] * dW_s[fwd_k])
      s_t[fwd_k] += ds_t[fwd_k]
      alpha_t[fwd_k] += g_t * s_t[fwd_k]
      # Zero absorbing boundary used for all the beta 
      # choices except beta = 0 and beta = 1
      if ((beta[fwd_k] > 0 and beta[fwd_k] < 1) and (F_t[fwd_k] <= 0)):
        F_t[fwd_k] = 0.0
      else:
        drift_F = (-g_t * s_t[fwd_k] * F_beta_t * shared_drift_part[fwd_k])
        F_t[fwd_k] += (drift_F * dt + alpha_t[fwd_k] * F_beta_t *dW_F[fwd_k])
        drift_correction[fwd_k - 1] = \
        corr_fwd_fwd[fwd_k - 1, fwd_k] * (tau * \
                                          g_t * s_t[fwd_k] * F_beta_t) / (1 + \
                                                                          tau * F_t[fwd_k])
                                          # Calculate the drift sum term
                                          # to be used by the next forward which will be simulated
        l = fwd_k
        while l <= (no_of_fwds - 1):
          shared_drift_part[fwd_k - 1] += drift_correction[l - 1]
          l += 1
      no_step += 1
    # Store the forwards simulated in the current Monte Carlo path
    for fwd_k in range(0, no_of_fwds):
      simulated_forwards[fwd_k, no_sim] = F_t[fwd_k]
    no_sim += 1
      
  return simulated_forwards

In [8]:
# Year fraction between two dates depending on day count convention chosen
# Reference SABR & SABR LIBOR Market Models, page 204, table Appendix A1

def yearFraction(date1, date2, dcc):
  '''
  Compute the year fraction between two dates, for the 
  same cases 30/360 and ACT/360
  
  @var date1: start date
  @var date2: end date
  @var dcc: string representing the day count convention
  '''
  # 30/360
  if dcc == "30/360":
    yearfrac = (360.0 * (date2.year - date1.year)
    + 30.0 * (date2.month - date1.month)
    + (date2.day - date1.day)) / 360.0
  # ACT/360
  elif dcc == "ACT/360":
    yearfrac = (date2.toordinal() - date1/toordinal()) / 360.0

  return yearfrac


In [11]:
# Business dates adjustment
# Reference SABR & SABR LIBOR Market Models, page 204, table Appendix A2

def proceedDays(term, date):
  '''
  @var term: string representing how to move the date on the calendar
  @var date: date to adjust
  '''
  if('D' in term.upper()):
    newDate = startDate.toordinal()
    newDate += int(term[:-1])
    newDate = datetime.date.fromordinal(newDate)
  else: 
    newDate = datetime.date(date.year, date.month, date.day)
  return newDate

def adjustmentBDate(date, bdc, ccy):
  '''
  @var date: date to adjust
  @var bdc: business date convention
  @var ccy: currency code
  '''
  # It is a Sunday
  if (date.isoweekday() == 7):
    if('FOL' in bdc.upper()):
      return adjustBDate(proceedDays("1D", date), bdc, ccy)
    else:
      return adjustBDate(proceedDays("-2D", date), bdc, ccy)
  # It is a Saturday
  elif (date.isoweekday()== 6):
    if("FOL" in bdc.upper()):
      return adjustBDate(proceedDays("2D", date), bdc, ccy)
    else:
      return adjustBDate(proceedDays("-1D", date), bdc, ccy)
  # It is a holiday. We use a HolidayCalendar object (code not displayed) which
  # contains all the holidays associated to London and the currency main
  # financial hubs
  elif(HolidayCalendar.isHoliday(date, ccy)):
    if('FOL' in bdc.upper()):
      return adjustBDate(proceedDays("1D", date), bdc, ccy)
    else:
      return adjustBDate(proceedDays("-1D", date), bdc, ccy)
  return date

In [13]:
# Possible constructor for a cash flow object. We introduced unadjusted period
# start and end dates as well, to see how the dates are adjusted. The year
# fraction  and any time periods on the time grid are calculated using the
# yearFraction Python function
# Reference SABR & SABR LIBOR Market Models, page 205, table Appendix A3

class CashFlowData(object):
  def __init__(self, asofdate, pStart, pEnd, payDate, unadEnd, unadStart, dcc):
    self._payDate = payDate
    # Day count convention
    self._dcc =  dcc
    # Period end
    self._periodEnd = pEnd
    # Unadjusted period end
    self._unadjustedPeriodEnd = unadEnd
    # Period start
    self._periodStart = pStart
    # Unadjusted period start
    self._unadjustedPeriodStart = unadStart
    self._yf = yearFraction(pStart, pEnd, dcc)
    asOfDate = asOfDate
    self._tToPay = yearFraction(asOfD
                                ate, payDate, None)
    self._tToStart = yearFraction(asOfDate, pStart, None)
    self._tToEnd = yearFraction(asOfDate, pEnd, None)
    